## Mask R-CNN - Inspect Trained Model

Code and visualizations to test, debug, and evaluate the Mask R-CNN model.

In [1]:
"""
Based on the work of Waleed Abdulla (Matterport)
Modified by github.com/GustavZ
"""

import os
import sys
import numpy as np
import tensorflow as tf


# Root directory of the project
ROOT_DIR = os.path.abspath("../")

# Import Mask RCNN
sys.path.append(ROOT_DIR)  # To find local version of the library

# Model  Directory 
MODEL_DIR = os.path.join(ROOT_DIR, "logs")

## Configurations

In [5]:
# MS COCO Dataset
import coco
config = coco.CocoConfig()
COCO_DIR = os.path.join(ROOT_DIR,"data/coco")

In [6]:
# Override the training configurations with a few
# changes for inferencing.
class InferenceConfig(config.__class__):
    # Run detection on one image at a time
    GPU_COUNT = 1
    IMAGES_PER_GPU = 1

config = InferenceConfig()
config.display()


Configurations:
BACKBONE                       mobilenetv1
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
DETECTION_MAX_INSTANCES        100
DETECTION_MIN_CONFIDENCE       0.7
DETECTION_NMS_THRESHOLD        0.3
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_MAX_DIM                  512
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  800
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              square
IMAGE_SHAPE                    [512 512   3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'mrcnn_mask_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'rpn_class_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE                     [28, 28]
MAX_GT_INSTANCES               100
MEAN_PIXEL                     [123.7 116.8 103.9]


## Notebook Preferences

In [7]:
# Device to load the neural network on.
# Useful if you're training a model on the same 
# machine, in which case use CPU and leave the
# GPU for training.
#DEVICE = "/cpu:0"
DEVICE = "/gpu:0"

# Inspect the model in training or inference modes
# values: 'inference' or 'training'
# TODO: code for 'training' test mode not ready yet
TEST_MODE = "inference"
#TEST_MODE = "training"

In [8]:
def get_ax(rows=1, cols=1, size=16):
    """Return a Matplotlib Axes array to be used in
    all visualizations in the notebook. Provide a
    central point to control graph sizes.
    
    Adjust the size attribute to control how big to render images
    """
    _, ax = plt.subplots(rows, cols, figsize=(size*cols, size*rows))
    return ax

## Load Model

In [9]:
# Local path to trained h5 weights file
MODEL_NAME = 'mask_rcnn_512_cocoperson_0396' # TODO: enter value here
H5_MODEL_PATH = os.path.join(MODEL_DIR, MODEL_NAME+".h5") # TODO: enter value here

In [10]:
import mmrcnn.model as modellib

# Create model in inference mode
with tf.device(DEVICE):
    model = modellib.MaskRCNN(mode=TEST_MODE, model_dir=MODEL_DIR,config=config)

# Set path to model weights
weights_path = H5_MODEL_PATH
#weights_path = model.find_last()[1]

# Load weights
print("Loading weights ", weights_path)
model.load_weights(weights_path, by_name=True)

('Loading weights ', '/home/gustav/workspace/Mobile_Mask_RCNN/logs/mask_rcnn_512_cocoperson_0396.h5')


## Prepare and Export Model

In [11]:
# name of the pb file we want to output
MODEL_NAME = 'mask_rcnn_512_cocoperson_0396' # TODO: enter value here

# Chose whether to quantize the graph
QUANTIZE = False

In [12]:
from tensorflow.tools.graph_transforms import TransformGraph
from tensorflow.python.framework import graph_util
from tensorflow.python.framework import graph_io
from keras import backend as K

# Get keras model and save
model_keras= model.keras_model
# All new operations will be in test mode from now on.
K.set_learning_phase(0)

# Create output layer with customized names
num_output = 7
pred_node_names = ["detections", "mrcnn_class", "mrcnn_bbox", "mrcnn_mask", "rois", "rpn_class", "rpn_bbox"]
pred_node_names = ["output_" + name for name in pred_node_names]
pred = [tf.identity(model_keras.outputs[i], name = pred_node_names[i])for i in range(num_output)]

# Get the object detection graph
sess = K.get_session()
if QUANTIZE:
    # Transformations
    transforms = ["quantize_weights", "quantize_nodes"]
    transformed_graph_def = TransformGraph(sess.graph.as_graph_def(), [], pred_node_names, transforms)
    constant_graph = graph_util.convert_variables_to_constants(sess, transformed_graph_def, pred_node_names)
    PB_MODEL_PATH = os.path.join(MODEL_DIR, MODEL_NAME+'.pb') 
else:
    constant_graph = graph_util.convert_variables_to_constants(sess, sess.graph.as_graph_def(), pred_node_names)
    PB_MODEL_PATH = os.path.join(MODEL_DIR, MODEL_NAME+'_quantized'+'.pb') 

INFO:tensorflow:Froze 217 variables.


INFO:tensorflow:Froze 217 variables.


Converted 217 variables to const ops.


In [14]:
# Write Output pb File
graph_io.write_graph(constant_graph, "/", PB_MODEL_PATH, as_text=False)

# Output Info
print('{} ops in the frozen graph.'.format(len(constant_graph.node)))
print('saved the freezed graph (ready for inference) at: ', PB_MODEL_PATH)

4139 ops in the frozen graph.
('saved the freezed graph (ready for inference) at: ', '/home/gustav/workspace/Mobile_Mask_RCNN/logs/mask_rcnn_512_cocoperson_0396.pb')


## Inference Model
Now, we can load the model from the pb file and then use it to infere

In [17]:
# DOES NOT WORK YET

import cv2 
from mmrcnn import utils

utils.set_cuda_visible_devices(config.GPU_COUNT)
with tf.device(DEVICE):
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        output_graph_def = tf.GraphDef()
        with open(PB_MODEL_PATH, "rb") as f:
            output_graph_def.ParseFromString(f.read())
            _ = tf.import_graph_def(output_graph_def, name="")

        image = cv2.imread(ROOT_DIR+"/ski.jpk")
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image,(config.IMAGE_MAX_DIM,config.IMAGE_MAX_DIM))
        image = np.expand_dims(image,0)

        image_resized, window, scale,_ = resize_image(image, min_dim=None, max_dim=None, padding=False)
        image_meta = compose_image_meta(image_id=1, image_shape=image_resized.shape, window=window, active_class_ids=[0,1])
        image_meta = np.expand_dims(image_meta,0)
        graph = tf.get_default_graph()
        input_image = graph.get_tensor_by_name("input_image:0")
        input_image_meta = graph.get_tensor_by_name("input_image_meta:0")
        mrcnn_mask = graph.get_tensor_by_name("output_mrcnn_mask:0")

        #[detections, mrcnn_class, mrcnn_bbox, mrcnn_mask,rois, rpn_class, rpn_bbox]
        feed = {input_image:image_resized, input_image_meta:image_meta}
        result = sess.run(mrcnn_mask, feed_dict = feed)
        print(result.shape)

InvalidArgumentError: Cannot assign a device for operation 'ROI/rpn_non_max_suppression/NonMaxSuppressionV2': Could not satisfy explicit device specification '/device:GPU:0' because no supported kernel for GPU devices is available.
Registered kernels:
  device='CPU'

	 [[Node: ROI/rpn_non_max_suppression/NonMaxSuppressionV2 = NonMaxSuppressionV2[_device="/device:GPU:0"](ROI/strided_slice_21, ROI/strided_slice_22, ROI/rpn_non_max_suppression/NonMaxSuppressionV2/max_output_size, ROI/rpn_non_max_suppression/iou_threshold)]]

Caused by op u'ROI/rpn_non_max_suppression/NonMaxSuppressionV2', defined at:
  File "/usr/lib/python2.7/runpy.py", line 174, in _run_module_as_main
    "__main__", fname, loader, pkg_name)
  File "/usr/lib/python2.7/runpy.py", line 72, in _run_code
    exec code in run_globals
  File "/usr/local/lib/python2.7/dist-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python2.7/dist-packages/traitlets/config/applicat